In [8]:
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
from envs.trading_env import TradingEnv
import ta

# === Step 1: Load raw CSV ===
def load_price_data(path):
    df = pd.read_csv(path)
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df = df.sort_values('Datetime').reset_index(drop=True)
    return df

# === Step 2: Add indicators ===
def add_technical_features(df):
    df['price_change'] = df['Close'].pct_change()
    df['volatility'] = df['Close'].rolling(window=10).std()
    df['range'] = (df['High'] - df['Low']) / df['Close']

    df['rsi'] = ta.momentum.RSIIndicator(close=df['Close'], window=14).rsi()
    macd = ta.trend.MACD(close=df['Close'])
    df['macd'] = macd.macd_diff()

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

# === Step 3: Setup dataframe ===
window_size = 50
csv_path = "./data/ETH-USD_test_data.csv"

df_raw = load_price_data(csv_path)
df = add_technical_features(df_raw)

usable_rows = len(df)
print(f"✅ Usable rows after indicators: {usable_rows}")

# Check that we can extract at least one window
if usable_rows <= window_size:
    raise ValueError(f"❌ Not enough data ({usable_rows}) after indicators for window size {window_size}")

# === Step 4: Create environment ===
feature_cols = ["price_change", "volatility", "range", "rsi", "macd"]
env = TradingEnv(df=df, window_size=window_size, initial_balance=10000, feature_cols=feature_cols)

# === Step 5: Load model ===
model = PPO.load("ppo_trading_bot_parallel.zip", env=env)

# === Step 6: Inference loop ===
obs, _ = env.reset()
done = False
total_reward = 0

while not done:
    action, _ = model.predict(obs)
    obs, reward, done, _, _ = env.step(action)
    total_reward += reward
    env.render()

print(f"\n🚀 Total Profit: {env.total_profit:.2f}")
print(f"💰 Final Balance: {env.balance:.2f}")


✅ Usable rows after indicators: 16959
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Step: 51, Balance: 10000.00, Profit: 0.00, Position: 1
Step: 52, Balance: 10000.00, Profit: 0.00, Position: 1
Step: 53, Balance: 10000.00, Profit: 0.00, Position: 1
Step: 54, Balance: 10000.85, Profit: 0.85, Position: 0
Step: 55, Balance: 10000.85, Profit: 0.85, Position: 1
Step: 56, Balance: 10000.85, Profit: 0.85, Position: 1
Step: 57, Balance: 9999.63, Profit: -0.37, Position: 0
Step: 58, Balance: 9999.63, Profit: -0.37, Position: 0
Step: 59, Balance: 9999.63, Profit: -0.37, Position: -1
Step: 60, Balance: 9999.82, Profit: -0.18, Position: 0
Step: 61, Balance: 9999.82, Profit: -0.18, Position: 0
Step: 62, Balance: 9999.82, Profit: -0.18, Position: 0
Step: 63, Balance: 9999.82, Profit: -0.18, Position: 0
Step: 64, Balance: 9999.82, Profit: -0.18, Position: -1
Step: 65, Balance: 9999.82, Profit: -0.18, Position: -1
Step: 66, Balance: 10001.07, Profit: 1.07, Position: 0
St